In [65]:
import pandas as pd
import numpy as np
import os
import glob
from tqdm import tqdm
%matplotlib inline

In [81]:
df_lists = sorted(glob.glob('../data/*.csv'))
print(f'all files are {len(df_lists)}')
df = None
for df_path in df_lists:
    if df is None:
        df = pd.read_csv(df_path)
    else:
        tmp = pd.read_csv(df_path)
        df = pd.concat([df,tmp])
df = df.reset_index(drop=True)


all files are 156


In [3]:
df.to_csv('../full_list.csv',index=None)

In [10]:
df[' protocol_name'].head(20)#.fillna('nan').str.lower().str.contains('|'.join(searchfor))

0     6.9 ThorAbd Liver Dynamic 3phase+Thorax
1     6.9 ThorAbd Liver Dynamic 3phase+Thorax
2     6.9 ThorAbd Liver Dynamic 3phase+Thorax
3     6.9 ThorAbd Liver Dynamic 3phase+Thorax
4     6.9 ThorAbd Liver Dynamic 3phase+Thorax
5     6.9 ThorAbd Liver Dynamic 3phase+Thorax
6     6.9 ThorAbd Liver Dynamic 3phase+Thorax
7     6.9 ThorAbd Liver Dynamic 3phase+Thorax
8     6.9 ThorAbd Liver Dynamic 3phase+Thorax
9                              5.10 ThorPel P
10                             5.10 ThorPel P
11                             5.10 ThorPel P
12                                  胸部　単純　SHR
13                                  胸部　単純　SHR
14                                  胸部　単純　SHR
15          ['胸部〜骨盤　膵ダイナミック3$BAj!', '(頸)胸部']
16          ['胸部〜骨盤　膵ダイナミック3$BAj!', '(頸)胸部']
17          ['胸部〜骨盤　膵ダイナミック3$BAj!', '(頸)胸部']
18          ['胸部〜骨盤　膵ダイナミック3$BAj!', '(頸)胸部']
19          ['胸部〜骨盤　膵ダイナミック3$BAj!', '(頸)胸部']
Name:  protocol_name, dtype: object

In [67]:
new_df = pd.DataFrame([])

df['ID'] = df['Facility_Code']+df[' Accesion_Number']
ids = df['ID'].unique()
##腹部以外のprotocol を排除
searchfor = ['abd','pancreas','liver','pk','pel','renal','kub','colon','骨盤','腹','体幹']
print(df.shape)
df = df[df[' protocol_name'].fillna('nan').str.lower().str.contains('|'.join(searchfor))]
print(df.shape)

df = df[df[' image_type'].str.lower().str.contains('axial')]
print(df.shape)
###スライス少ないのはスカウト等なので排除
###window_centerがマイナスなのは肺野なので排除。それと一緒に取っている同じスライス数の画像もおそらく胸部の縦隔条件なので排除。

for id in tqdm(ids):
    tmp = df[df['ID'] == id].reset_index(drop=True)
    num_slices = tmp[' Number_of_Slices']
    window_isminus = tmp[' window_center'].str.contains('-')
    num_slices_low = num_slices<40
    lung_slices = num_slices[window_isminus]
    lung_slices = np.isin(num_slices,lung_slices)

    exclude_slices = num_slices_low | lung_slices
    

    tmp = tmp[~exclude_slices].reset_index(drop=True)

    new_df = pd.concat([new_df,tmp])

new_df = new_df.reset_index(drop=True)
new_df['path'] = new_df['Facility_Code'].astype('str')+'_'+new_df[' Accesion_Number'].astype('int').astype('str')+\
    '_'+new_df[' Series_Number'].fillna(9999).astype('int').astype('str').str.zfill(4)+'_'+new_df[' Number_of_Slices'].astype(int).astype('str').str.zfill(3)+'.nii.gz'

(218034, 18)
(93131, 18)
(59128, 18)


100%|██████████| 52231/52231 [03:47<00:00, 229.75it/s]


In [30]:

new_df.to_csv('../include_abdomen.csv',index=False,encoding='utf_8_sig')
new_df['path'].to_csv('../include_abdomen_path.csv',index=False,encoding='utf_8_sig')

In [71]:
new_df['path'].value_counts()

43157400144_1776520420220118_0009_420.nii.gz      4
43157400144_1776520420220118_0007_420.nii.gz      4
43157400144_1776520420220118_0010_420.nii.gz      4
17035395872_1210200375998535_0301_5824.nii.gz     4
17035395872_1210200375998535_0302_16768.nii.gz    4
                                                 ..
17144592271_4590839120220122_0002_138.nii.gz      1
17144592271_4603555320220127_0002_057.nii.gz      1
17144592271_4604165520220127_0002_050.nii.gz      1
17144592271_4604165520220127_0003_043.nii.gz      1
118_6010018780460_0201_158.nii.gz                 1
Name: path, Length: 41190, dtype: int64

In [57]:
df = pd.read_csv('../rename.csv')
new_df = new_df.merge(df,how='left',right_on='original_name',left_on='path')#.to_csv('rename.csv',index=False)

In [46]:
sm = 0
for i in np.arange(0,450,50):
    tmp = new_df[(new_df[' Number_of_Slices']>=i) & (new_df[' Number_of_Slices']<i+50)]
    print(i,i+50,len(tmp))
    sm += len(tmp)

0 50 2741
50 100 17843
100 150 15994
150 200 5494
200 250 356
250 300 168
300 350 139
350 400 26
400 450 28


In [120]:
np.arange(15,450,1)

array([ 15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 19

In [47]:
###スライス数ごとに画像を分けて、別々のフォルダに保存する
###フォルダ内のファイル数を一定に保つ


import pandas as pd
import glob
import os
import shutil
import argparse
from tqdm import tqdm
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='In order to remove unnecessary background, crop images based on segmenation labels.')
    parser.add_argument('--datadir', default="../data/abd_20220101-20220605/",
                        help='dafault data directory')
    parser.add_argument('--start_id', default=0, type=int,
                        help='a number to start rename , (default: 0)')
    parser.add_argument('--num_threads', default=20, type=int,
                        help='number of threds to process , (default: 20)')

    args = parser.parse_args()
    print(args)
    datadir = args.datadir
    start_id = args.start_id

    img_paths = sorted(glob.glob(os.path.join(datadir,'*.nii.gz')))
    print(len(img_paths))
    num_imgs = len(img_paths)
    num_slices = [int(i.split('_')[-1].split('.')[0]) for i in img_paths]

    new_paths = [f'jmid_{str(i+start_id).zfill(7)}_0000.nii.gz' for i in range(num_imgs)]
    img_paths_base = [os.path.basename(p) for p in img_paths]

    df = pd.DataFrame([img_paths_base,new_paths]).T
    df.columns = ['original_name','renamed']
    df.to_csv('rename_20211001-20211231.csv',index=False)

    dir_id = 0

    for i in range(9):
        os.makedirs(f'../data1/remain_{str(i)}_0',exist_ok=True)

    for num_slice,img_path,new_path in tqdm(zip(num_slices,img_paths,new_paths)):
        if num_slice>=450:continue
        else:
            out_dir = '../data1/remain_' + str(8-i//50) + '_0'
            shutil.copyfile(img_path,os.path.join(out_dir,new_path))
    print('data copied!')
    ####
    ####8,7→5000 6→2000, 5→1000 4→700 3→400 2→300 1,0→100
    for i in range(9):
        if i == 8 or i == 7:
            upper = 5000
        elif i == 6:
            upper = 2000
        elif i == 5:
            upper = 1000
        elif i == 4:
            upper = 700
        elif i == 3:
            upper = 400
        elif i == 2:
            upper = 300
        else:
            upper = 100


        files = glob.glob(f'../data1/remain_{str(i)}_0/*nii.gz')
        num_files = len(files)
        num_folder = num_files//upper
        for n in range(num_folder):
            os.makedirs(f'../data1/remain_{str(i)}_{str(n+1)}',exist_ok=True)
        for k,path in enumerate(files):
            dir_id = k // upper
            if dir_id >= 1:
                shutil.move(path,os.path.join(f'../data1/remain_{str(i)}_{str(dir_id)}',os.path.basename(path)))
    print('file moved.')

##########################################3333

df = pd.read_csv('merged_rename.csv')
df.drop_duplicates(subset='renamed', keep='last', inplace=True)
remain_paths = glob.glob('../data/remain_img/*.nii.gz')
remain_base_paths = [os.path.basename(p) for p in remain_paths]
num_processed_imgs = 0
num_dir = 0

for i in range(9):
    os.makedirs(f'../data/remain_{str(i)}_0',exist_ok=True)

for i in np.arange(0,450,50):
    print(i)
    tmp = df[(df[' Number_of_Slices']>=i) & (df[' Number_of_Slices']<i+50)]
    for n,name in tqdm(enumerate(tmp['renamed'])):
        if name in remain_base_paths:
            num_processed_imgs+=1
            dirsize = str(9-i//50)
            out_dir = '../data/remain_'+dirsize+str(num_dir)
            shutil.copyfile(f'../data/remain_img/{name}',out_dir)
        if len(os.listdir(out_dir))>=2000:
            num_dir+=1
            os.makedirs(f'../data/remain_{dirsize}_{num_dir}',exist_ok=True)
    num_dir = 0

print(num_processed_imgs)


42789

In [80]:
new_df

,Facility_Code,Accesion_Number,Series_Number,Number_of_Slices,modality,slice_thickness,patient_sex,patient_age,contrast_bolus_agent,manufacturer,model_name,conv_kernel,window_center,window_width,patient_orientation,image_type,protocol_name,ID,path,path_
0,43157400144,1148655720201211.0,2.0,53,CT,5.0,M,074Y,NaN,GE MEDICAL SYSTEMS,Discovery CT750 HD,STANDARD,35,300,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",6.9 ThorAbd Liver Dynamic 3phase+Thorax,1148698877601355.0,43157400144_1148655720201211_0002_053.nii.gz,43157400144_1148655720201211_0002
1,43157400144,1148655720201211.0,3.0,53,CT,5.0,M,074Y,NaN,GE MEDICAL SYSTEMS,Discovery CT750 HD,STANDARD,55,320,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",6.9 ThorAbd Liver Dynamic 3phase+Thorax,1148698877601355.0,43157400144_1148655720201211_0003_053.nii.gz,43157400144_1148655720201211_0003
2,43157400144,1148655720201211.0,4.0,53,CT,5.0,M,074Y,NaN,GE MEDICAL SYSTEMS,Discovery CT750 HD,STANDARD,55,320,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",6.9 ThorAbd Liver Dynamic 3phase+Thorax,1148698877601355.0,43157400144_1148655720201211_0004_053.nii.gz,43157400144_1148655720201211_0004
3,43157400144,1148655720201211.0,5.0,53,CT,5.0,M,074Y,NaN,GE MEDICAL SYSTEMS,Discovery CT750 HD,STANDARD,55,320,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",6.9 ThorAbd Liver Dynamic 3phase+Thorax,1148698877601355.0,43157400144_1148655720201211_0005_053.nii.gz,43157400144_1148655720201211_0005
4,43157400144,1167702520201223.0,2.0,131,CT,5.0,F,072Y,NaN,GE MEDICAL SYSTEMS,Discovery CT750 HD,STANDARD,30,300,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",5.10 ThorPel P,1167745677601367.0,43157400144_1167702520201223_0002_131.nii.gz,43157400144_1167702520201223_0002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42819,118,6010016870769.0,201,131,CT,5.0,F,034Y,NaN,Philips,iCT 256,B,"[40, 40]","[340, 340]",NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",Chest-Pelivis /iCT in HU,6010016870887.0,118_6010016870769_0201_131.nii.gz,118_6010016870769_0201
42820,118,6010018198758.0,201,148,CT,5.0,F,062Y,NaN,Philips,iCT 256,B,"[40, 40]","[340, 340]",NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",Neck-Pelivis /iCT in HU,6010018198876.0,118_6010018198758_0201_148.nii.gz,118_6010018198758_0201
42821,118,6010018748732.0,301,164,CT,5.0,M,076Y,NaN,Philips,iCT 256,B,"[40, 40]","[300, 300]",NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",Neck-Pelivis /iCT in HU,6010018748850.0,118_6010018748732_0301_164.nii.gz,118_6010018748732_0301
42822,118,6010018776642.0,201,170,CT,5.0,M,082Y,NaN,Philips,iCT 256,B,"[40, 40]","[340, 340]",NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",Neck-Pelivis /iCT in HU,6010018776760.0,118_6010018776642_0201_170.nii.gz,118_6010018776642_0201


In [49]:
new_df[new_df[' Number_of_Slices']>450].shape


(35, 19)

In [ ]:
import os
import shutil

data_dir = '../data/finish_pred/'
finished_list = []
finished_paths = sorted(glob.glob(os.path.join(data_dir,'*.nii.gz')))
for path in finished_paths:
    img_path = path.split('.')[0]+'_0000.nii.gz'
    finished_list.append(img_path)

pd.DataFrame(finished_list).T.to_csv('finished.csv',index=None)


In [162]:
import logging
import os
import sys
import shutil
import tempfile

import matplotlib.pyplot as plt
import torch
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import time
import pathlib

from torch.utils.data import default_collate

import monai
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import DataLoader, ImageDataset, PersistentDataset, pad_list_data_collate, ThreadBuffer
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    RandRotate90,
    Resize,
    ScaleIntensity,
)
from monai.transforms import (
    EnsureChannelFirstd,
    AsDiscrete,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
)
from monai.utils import get_torch_version_tuple, set_determinism
pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print_config()

MONAI version: 1.0.0
Numpy version: 1.21.5
Pytorch version: 1.11.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 170093375ce29267e45681fcec09dfa856e1d7e7
MONAI __file__: /home/jubuntu/anaconda3/envs/monai/lib/python3.8/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.8
Nibabel version: 3.2.2
scikit-image version: 0.19.2
Pillow version: 9.0.1
Tensorboard version: 2.6.0
gdown version: 4.4.0
TorchVision version: 0.12.0
tqdm version: 4.64.0
lmdb version: 1.2.1
psutil version: 5.8.0
pandas version: 1.4.2
einops version: 0.4.1
transformers version: 4.14.1
mlflow version: 1.25.1
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [159]:
images = glob.glob('../data/abd_20220101-20220605/118_*.nii.gz')[:20]
# 2 binary labels for gender classification: man or woman
labels = np.array([0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0])

# Represent labels in one-hot format for binary classifier training,
# BCEWithLogitsLoss requires target to have same shape as input
labels = torch.nn.functional.one_hot(torch.as_tensor(labels)).float()

data_dicts = [
    {"image": image_name, "label": label_name}
    for image_name, label_name in zip(images, labels)
]

amp = True
use_buffer = True

In [175]:
##cacheする場所を設定する

persistent_cache = pathlib.Path(os.getcwd(), "persistent_cache")
persistent_cache.mkdir(parents=True, exist_ok=True)



# Define transforms
train_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize((96, 96, 96)) ])

train_transforms = Compose(
        [
            LoadImaged(keys=["image"]),
            EnsureChannelFirstd(keys=["image"]),
            Orientationd(keys=["image"], axcodes="RAS"),
            ScaleIntensityRanged(
                keys=["image"],
                a_min=-57,
                a_max=164,
                b_min=0.0,
                b_max=1.0,
                clip=True,
            ),
        ]
    )

val_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize((96, 96, 96))])

#data = default_collate([images,labels])

# Define nifti dataset, data loader
check_ds = PersistentDataset(
    data=data_dicts[:10], transform=train_transforms, cache_dir=persistent_cache
)
check_loader = DataLoader(check_ds, batch_size=1, num_workers=2, pin_memory=pin_memory,collate_fn=pad_list_data_collate,)

batch= monai.utils.misc.first(check_loader)
im,label = batch['image'],batch['label']
print(type(im), im.shape, label, label.shape)

# create a training data loader

train_ds = PersistentDataset(
    data=data_dicts[:10], transform=train_transforms, cache_dir=persistent_cache
)
#train_ds = ImageDataset(image_files=images[:10], labels=labels[:10], transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=2, pin_memory=pin_memory, collate_fn=pad_list_data_collate,)

# create a validation data loader
val_ds = PersistentDataset(
    data=data_dicts[:-10], transform=val_transforms, cache_dir=persistent_cache
)
#val_ds = ImageDataset(image_files=images[-10:], labels=labels[-10:], transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=2, num_workers=2, pin_memory=pin_memory, collate_fn=pad_list_data_collate,)

<class 'monai.data.meta_tensor.MetaTensor'> (1, 1, 512, 512, 154) tensor([[1., 0.]]) torch.Size([1, 2])


In [176]:
next(iter(check_loader))['image'].shape

(1, 1, 512, 512, 154)

In [178]:
# Create DenseNet121, CrossEntropyLoss and Adam optimizer
model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=1, out_channels=2).to(device)

loss_function = torch.nn.CrossEntropyLoss()
# loss_function = torch.nn.BCEWithLogitsLoss()  # also works with this data

optimizer = torch.optim.Adam(model.parameters(), 1e-4)
scaler = torch.cuda.amp.GradScaler() if amp else None

# start a typical PyTorch training
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
writer = SummaryWriter()
max_epochs = 5
set_determinism(seed=0)

for epoch in range(max_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss = 0
    step = 0
    src = ThreadBuffer(train_loader, 1) if use_buffer else train_loader
    for batch_data in src:
        step_start = time.time()
        step += 1
        inputs, labels = batch_data['image'].to(device), batch_data['label'].to(device)
        optimizer.zero_grad()

        if amp and scaler is not None:
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = loss_function(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
        epoch_loss += loss.item()
        print(
                f"{step}/{len(train_ds) // train_loader.batch_size},"
                f" train_loss: {loss.item():.4f}"
                f" step time: {(time.time() - step_start):.4f}"
        )
        epoch_len = len(train_ds) // train_loader.batch_size

        # outputs = model(inputs)
        # loss = loss_function(outputs, labels)
        # loss.backward()
        # optimizer.step()
        # epoch_loss += loss.item()
        # epoch_len = len(train_ds) // train_loader.batch_size
        # print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
        writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)

    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()

        num_correct = 0.0
        metric_count = 0
        for val_data in val_loader:
            val_images, val_labels = val_data['image'].to(device), val_data['label'].to(device)
            if amp:
                with torch.cuda.amp.autocast():
                    val_outputs = model(val_images)
                    value = torch.eq(val_outputs.argmax(dim=1), val_labels.argmax(dim=1))
                    metric_count += len(value)
                    num_correct += value.sum().item()
            else:
                with torch.no_grad():
                    val_outputs = model(val_images)
                    value = torch.eq(val_outputs.argmax(dim=1), val_labels.argmax(dim=1))
                    metric_count += len(value)
                    num_correct += value.sum().item()
                    
        metric = num_correct / metric_count
        metric_values.append(metric)

        if metric > best_metric:
            best_metric = metric
            best_metric_epoch = epoch + 1
            torch.save(model.state_dict(), "best_metric_model_classification3d_array.pth")
            print("saved new best metric model")

        print(f"Current epoch: {epoch+1} current accuracy: {metric:.4f} ")
        print(f"Best accuracy: {best_metric:.4f} at epoch {best_metric_epoch}")
        writer.add_scalar("val_accuracy", metric, epoch + 1)

print(f"Training completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
writer.close()

----------
epoch 1/5
1/5, train_loss: 0.6777 step time: 0.7180
2/5, train_loss: 0.5593 step time: 1.3357
3/5, train_loss: 0.5142 step time: 1.1398
4/5, train_loss: 0.7891 step time: 0.5784
5/5, train_loss: 0.9563 step time: 1.2021
epoch 1 average loss: 0.6993
----------
epoch 2/5
1/5, train_loss: 0.4247 step time: 1.0870
2/5, train_loss: 0.6340 step time: 0.8471
3/5, train_loss: 0.6943 step time: 0.5727


RuntimeError: CUDA out of memory. Tried to allocate 688.00 MiB (GPU 0; 23.70 GiB total capacity; 15.55 GiB already allocated; 129.31 MiB free; 21.04 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [145]:
next(iter(train_loader))

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/jubuntu/anaconda3/envs/monai/lib/python3.8/site-packages/monai/transforms/transform.py", line 90, in apply_transform
    return [_apply_transform(transform, item, unpack_items) for item in data]
  File "/home/jubuntu/anaconda3/envs/monai/lib/python3.8/site-packages/monai/transforms/transform.py", line 90, in <listcomp>
    return [_apply_transform(transform, item, unpack_items) for item in data]
  File "/home/jubuntu/anaconda3/envs/monai/lib/python3.8/site-packages/monai/transforms/transform.py", line 55, in _apply_transform
    return transform(parameters)
  File "/home/jubuntu/anaconda3/envs/monai/lib/python3.8/site-packages/monai/transforms/intensity/array.py", line 455, in __call__
    ret = rescale_array(img_t, self.minv, self.maxv, dtype=self.dtype)
  File "/home/jubuntu/anaconda3/envs/monai/lib/python3.8/site-packages/monai/transforms/utils.py", line 184, i